In [1]:
import json
import pandas as pd
import os
from collections import defaultdict

In [129]:
class Mapping():
    def __init__(self):
        self.mapping_dict = self.get_mapping_dict()
        
    def get_mapping_dict(self):
        with open('json file/mapping_dict_lv3.json', 'r') as jf:
            mapping_dict = json.load(jf)
        return mapping_dict
        
    def mapping(self, dataset, fetch='ALL'):
        if fetch =='ALL':
            return self.mapping_dict[dataset]
        elif fetch not in self.mapping_dict[dataset]:
            raise Exception("Sorry, this is not normal item!")
        else:
            return self.mapping_dict[dataset][fetch]

In [130]:
a = Mapping()

## pipline

In [131]:
def get_mapping_df(orign_df, mapping_dict):
    mapped_dict = defaultdict(list)
    if 'geo_accession' not in orign_df: return
    mapped_dict['ID'] = orign_df['geo_accession']
    for key, values in mapping_dict.items():
        # key: 'Disease'
        # values: {'status(0=healthy,1=cancer):ch1': defaultdict(dict, {0: 'Healthy', 1: 'Breast cancer'})},
        for value_key, value_mapping in values.items():
            # if key =='Age':
            #     print(value_key, value_mapping, list(orign_df[value_key]))
            if value_key in orign_df:
                # print(mapping_dict[key][value_key])
                orign_df[value_key] = orign_df[value_key].astype('str')
                orign_df[value_key] = orign_df[value_key].replace({'nan': 'NaN', '': 'NaN'})
                # print(list(orign_df[value_key]))
                mapped_dict[key] = orign_df[value_key].replace(mapping_dict[key][value_key])
                # print(list(mapped_dict[key]))
            else:
                mapped_dict[key] = [value_key]*orign_df.shape[0]
    mapped_df = pd.DataFrame(mapped_dict)
    for col in list(mapped_df):
        mapped_df = mapped_df.drop(mapped_df[mapped_df[col] == '#remove#'].index)
    # mapped_df = mapped_df.drop(mapped_df[mapped_df.Disease == '#remove#'].index)
    return mapped_df

In [132]:
dir_path = '../Datasets/orignal'
for i, file_name in enumerate(os.listdir(dir_path)):
    file_path = os.path.join(dir_path, file_name)
    if file_name.split('.')[-1] == 'csv':
        data = pd.read_csv(file_path)
    if file_name.split('.')[-1] == 'xlsx':
        data = pd.read_excel(file_path)
    dataset = file_name.split('_')[0]
    mapping_dict = a.mapping(dataset, fetch='ALL')
    mapped_df = get_mapping_df(data, mapping_dict)
    try:
        mapped_df.to_csv(os.path.join('../Datasets/mapped_lv3', file_name.split('.')[0]+'.csv'), index=None)
    except:
        continue